### Text generation using Project Gutenberg

In this tutorial, we will download a story from Project Gutenberg and use it to feed a text generation model.

About Project Gutenberg:
"Project Gutenberg offers over 56,000 free eBooks: Choose among free epub books, free kindle books, download them or read them online. You will find the world's great literature here, especially older works for which copyright has expired. We digitized and diligently proofread them with the help of thousands of volunteers."

Navigate to https://www.gutenberg.org/ and pick a book with a plain text file of around 200 kb (Alice in Wonderland is a good option: https://www.gutenberg.org/ebooks/11). Download the text as the basis for your model.

### Develop a Small LSTM Recurrent Neural Network
In this section we will develop a simple LSTM network to learn sequences of characters from Alice in Wonderland. In the next section we will use this model to generate new sequences of characters.

Let’s start off by importing the classes and functions we intend to use to train our model.

In [42]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, GRU
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Next, we need to load the ASCII text for the book into memory and convert all of the characters to lowercase to reduce the vocabulary that the network must learn.

In [46]:
# load ascii text and covert to lowercase
import re
filename = "./alice.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()
#raw_text = re.sub(r':',' ', raw_text)

Now that the book is loaded, we must prepare the data for modeling by the neural network. We cannot model the characters directly, instead we must convert the characters to integers.

We can do this easily by first creating a set of all of the distinct characters in the book, then creating a map of each character to a unique integer.

In [74]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

You can see that there may be some characters that we could remove to further clean up the dataset that will reduce the vocabulary and may improve the modeling process.

Now that the book has been loaded and the mapping prepared, we can summarize the dataset.

In [75]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  163781
Total Vocab:  59


We now need to define the training data for the network. There is a lot of flexibility in how you choose to break up the text and expose it to the network during training.

In this tutorial we will split the book text up into subsequences with a fixed length of 100 characters, an arbitrary length. We could just as easily split the data up by sentences and pad the shorter sequences and truncate the longer ones.

Each training pattern of the network is comprised of 100 time steps of one character (X) followed by one character output (y). When creating these sequences, we slide this window along the whole book one character at a time, allowing each character a chance to be learned from the 100 characters that preceded it (except the first 100 characters of course).

In [76]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  163681


Now that we have prepared our training data we need to transform it so that it is suitable for use with Keras.

First we must transform the list of input sequences into the form [samples, time steps, features] expected by an LSTM network.

Next we need to rescale the integers to the range 0-to-1 to make the patterns easier to learn by the LSTM network that uses the sigmoid activation function by default.

Finally, we need to convert the output patterns (single characters converted to integers) into a one hot encoding. This is so that we can configure the network to predict the probability of each of the different characters in the vocabulary (an easier representation) rather than trying to force it to predict precisely the next character. Each y value is converted into a sparse vector, full of zeros except with a 1 in the column for the letter (integer) that the pattern represents.


In [77]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

We can now define our LSTM model. Here we define a single hidden LSTM layer with 256 memory units. The network uses dropout with a probability of 20. The output layer is a Dense layer using the softmax activation function to output a probability prediction for each of the characters between 0 and 1.

The problem is really a single character classification problem with n character classes and as such is defined as optimizing the log loss (cross entropy), here using the ADAM optimization algorithm for speed.


In [78]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

There is no test dataset. We are modeling the entire training dataset to learn the probability of each character in a sequence.

We are not interested in the most accurate (classification accuracy) model of the training dataset. This would be a model that predicts each character in the training dataset perfectly. Instead we are interested in a generalization of the dataset that minimizes the chosen loss function. We are seeking a balance between generalization and overfitting but short of memorization.


In [81]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

model.fit(X, y, epochs=30, batch_size=256, callbacks=callbacks_list)

Epoch 1/30
163681/163681 [==============================] - 325s 2ms/step - loss: 2.7612

Epoch 00001: loss improved from inf to 2.76121, saving model to weights-improvement-01-2.7612.hdf5
Epoch 2/30
163681/163681 [==============================] - 326s 2ms/step - loss: 2.7099

Epoch 00002: loss improved from 2.76121 to 2.70990, saving model to weights-improvement-02-2.7099.hdf5
Epoch 3/30
163681/163681 [==============================] - 325s 2ms/step - loss: 2.6557

Epoch 00003: loss improved from 2.70990 to 2.65570, saving model to weights-improvement-03-2.6557.hdf5
Epoch 4/30
163681/163681 [==============================] - 325s 2ms/step - loss: 2.6133

Epoch 00004: loss improved from 2.65570 to 2.61331, saving model to weights-improvement-04-2.6133.hdf5
Epoch 5/30
163681/163681 [==============================] - 325s 2ms/step - loss: 2.5736

Epoch 00005: loss improved from 2.61331 to 2.57358, saving model to weights-improvement-05-2.5736.hdf5
Epoch 6/30
163681/163681 [=============

After running the example, you should have a number of weight checkpoint files in the local directory.

You can delete them all except the one with the smallest loss value, eg `weights-improvement-19-1.9435.hdf5`

### Generating Text with an LSTM Network
Generating text using the trained LSTM network is relatively straightforward.

Firstly, we load the data and define the network in exactly the same way, except the network weights are loaded from a checkpoint file and the network does not need to be trained.


In [91]:
# load the network weights
filename = "./text_gen_weights_new/weights-improvement-10-2.3837.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

Also, when preparing the mapping of unique characters to integers, we must also create a reverse mapping that we can use to convert the integers back to characters so that we can understand the predictions.


In [83]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

The simplest way to use the Keras LSTM model to make predictions is to first start off with a seed sequence as input, generate the next character then update the seed sequence to add the generated character on the end and trim off the first character. This process is repeated for as long as we want to predict new characters (e.g. a sequence of 1,000 characters in length).

We can pick a random input pattern as our seed sequence, then print generated characters as we generate them.


In [93]:
import sys
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print( "Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
"  too flustered to tell you--all i know is, something comes at me
like a jack-in-the-box, and up i go "
 woe toile toe touee toen i mene toe toule toen i men to toe tou do toe mort oo the toite '

'io sou toen tou do wou doon ' said the mone turtled  bnd the woile tas io toe toile  and the toene to toe toele to the toele  and the toene tas io toe toile the woile toen the woule the toele to the toele  and the woine tas io toe toile the woile toen the woule toen the woole toen the woole toen the woule the woele toe toele the woele toe woele the woele the woene toe woele the woele the woene tae toe toile the woele the woene the woene the woene the woene the woene the woene the woene the woene tae toe toile the woele the woene the woene the woene the woene the woene the woene the woene the woene tae toe toile the woele the woene the woene the woene the woene the woene the woene the woene the woene tae toe toile the woele the woene the woene the woene the woene the woene the woene th

What about the results shows good language generation? What is done poorly?

### Exercise 1: Try a bigger model

We will keep the number of memory units the same at 256, but add a second layer.

In [69]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

Again, what about the results shows good language generation? What is done poorly?

### Exercise 2: Create a word based model

Instead of generating language character by character, let's build an word generating model.

Use the RNN code from the previous notebook as a guide!